## Model 3
A Naive Bayes approach...

In [34]:
import numpy as np 
import pandas as pd 
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

train_df = pd.read_csv("train.csv")
test_df=pd.read_csv("test.csv")
train_df.head()

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


In [35]:
train_df['author_num'] = train_df.author.map({'EAP':0, 'HPL':1, 'MWS':2})
X = train_df['text'].tolist()
y = train_df['author_num'].tolist()

In [36]:
print(X[0])

This process, however, afforded me no means of ascertaining the dimensions of my dungeon; as I might make its circuit, and return to the point whence I set out, without being aware of the fact; so perfectly uniform seemed the wall.


### In this model the only preprocessing that occurs is  dropping stop words. I want to keep this model simple to compare the results to my Model 1, which had more preprocessing.
##### * In Model 1 we dropped stop words, separated punctuation from words, shortened the training sentences to 500 characters and removed lower frequency words.

In [43]:
# Drop stop words
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
stops = set(stopwords.words('english'))

# now we will use the text and author_id fields to train a classifier.
#  We have to: 
#  1. Get the sentences, 
sents = X #sentence
labels = y #author
#  2. Tokenize each sentence, 
tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(sents)
sequences = tokenizer.texts_to_sequences(sents)
print(len(sequences))
print(sequences[0])
##    Get a vector of unique terms here
print('Found %s unique tokens before stopwords removal.' % len(tokenizer.word_index))
print([w for w in tokenizer.word_index.items()][:5])
word_index = dict([(w,i) for w,i in tokenizer.word_index.items() if w not in stops])
print('Found %s unique tokens after stopwords removal.' % len(word_index))

19579
[26, 2945, 143, 1372, 22, 36, 294, 2, 7451, 1, 2440, 2, 10, 4556, 16, 6, 79, 179, 48, 4245, 3, 295, 4, 1, 249, 1943, 6, 326, 74, 134, 123, 891, 2, 1, 313, 39, 1438, 4928, 98, 1, 430]
Found 25943 unique tokens before stopwords removal.
[('the', 1), ('of', 2), ('and', 3), ('to', 4), ('a', 5)]
Found 25808 unique tokens after stopwords removal.


In [44]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

### tokenization and vectorization to save the count of each word in document term matrix.
A document-term-matrix describes the frequency of terms that occur in a collection. Knowing frequent words can be used a feature.

In [45]:
vect = CountVectorizer(lowercase=False, token_pattern=r'\w+|\,')
x_cv=vect.fit_transform(X)
x_train_cv = vect.transform(x_train)
x_test_cv = vect.transform(x_test)

In [47]:
mnb = MultinomialNB()
mnb.fit(x_train_cv, y_train)
print(mnb.score(x_test_cv, y_test))
predict = mnb.predict_proba(x_test_cv)

0.846527068437


#### Model 3 score's is about the same as Model 1 which was surprising. I thought it would of scored lower. But complicating things doesn't mean you'll get better results. I saw this in Model 1 where adding more layers only made the results worse. 

In [ ]:
result=pd.DataFrame()
result["id"]=testing_data["id"]
result["EAP"]=predict[:,0]
result["HPL"]=predict[:,1]
result["MWS"]=predict[:,2]
result.head()
result.to_csv("model_3_sub.csv", index=False)

#### Post Kaggle Submission #2:
While it had a similar accuracy, it scored worse than Model 1's submission, which was what I was initially expecting.

###### Model 1: 0.36295
###### Model 3: 0.47007